In [1]:
filename = "out.txt"

In [2]:
with open(filename, 'rb') as f:
    lines = f.readlines()

In [3]:
for line in lines:
    print(str(line))

b'Making lexicon from standard input...done! (took 0.000706966 seconds)\n'
b'Lexicon has 31 nodes and 34 edges\n'
b'k : 1\n'
b'i : 1\n'
b'h : 1\n'
b'w : 4\n'
b'u : 1\n'
b'e : 2\n'
b'a : 2\n'
b'c : 2\n'
b'g : 4\n'
b'. : 5\n'
b'l : 3\n'
b'm : 2\n'
b'o : 6\n'
b'3\n'
b'4\n'
b'Writing lexicon to standard output...q\x91\xf5\xba\xea\xeb\x8e\xef\xab\xae{\xbe\xef\xb9\xee\xcb\x96\xae\xbd\xef{\xdfp\xb9\xd0]\xf7m]s\xdd\xf7}\xcfv\\\xb5u\xdb\xf7u\xc3t\xdd\x8fdone! (took 0.00026736 seconds)\n'


In [4]:
#calculate the number of bits from all the edges
print(lines[1])
print(lines[1].split()[-2])
edges = int(lines[1].split()[-2])
print(edges)
bits_per_edge = 3
print("{0} bits from edges".format(bits_per_edge * edges))

b'Lexicon has 31 nodes and 34 edges\n'
b'34'
34
102 bits from edges


In [7]:
#format into hist
hist = {}
for line in lines:
    if ':' in str(line): #makes sure it is a line with data
        line = str(line[:-1])
        symbol, occ = line.split(" : ")
        hist[str(symbol[2:])] = int(occ[:-1])

In [20]:
print(len(hist.keys()))

13


In [38]:
#calculate entropy of the set I want to use
frequency_cutoff = 3
below_cutoff = 0
label_num = 0
total_counts = 0
total_bits_below_cutoff = 0
frequencies = []
for symbol, occ in hist.items():
    if occ >= frequency_cutoff:
        print(symbol, occ)
        label_num += 1
        frequencies.append(occ)
        total_counts += occ
    else:
        below_cutoff += 1
        total_bits_below_cutoff += 7 * (len(symbol) + 1)
for i in range(len(frequencies)):
    frequencies[i] /= total_counts
print(frequencies)
entropy = 0
for f in frequencies:
    entropy -= f * math.log(f, 2)
print(entropy)
# entropy = label_num / total_counts #redo this!!!
print("the entropy is {0} with {1} edges, taking {2} bits".format(entropy, edges, bits_per_edge * edges * entropy))
#this willl need to be changed in the future, assumes each label is 1 character long
print("there are {0} labels with less than {1} occurences, which use {2} bits".format(below_cutoff, frequency_cutoff, total_bits_below_cutoff))
total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + total_bits_below_cutoff
print("{0} total bits with a {1} frequency cutoff".format(int(total_bits), frequency_cutoff))

w 4
g 4
. 5
l 3
o 6
[0.18181818181818182, 0.18181818181818182, 0.22727272727272727, 0.13636363636363635, 0.2727272727272727]
2.28332693768606
the entropy is 2.28332693768606 with 34 edges, taking 232.89934764397813 bits
there are 8 labels with less than 3 occurences, which use 112 bits
446 total bits with a 3 frequency cutoff


In [40]:
#param sweep
for frequency_cutoff in range(6):
    #calculate entropy of the set I want to use
    below_cutoff = 0
    label_num = 0
    total_counts = 0
    total_bits_below_cutoff = 0
    frequencies = []
    for symbol, occ in hist.items():
        if occ >= frequency_cutoff:
#             print(symbol, occ)
            label_num += 1
            frequencies.append(occ)
            total_counts += occ
        else:
            below_cutoff += 1
            total_bits_below_cutoff += 7 * (len(symbol) + 1)
    for i in range(len(frequencies)):
        frequencies[i] /= total_counts
#     print(frequencies)
    entropy = 0
    for f in frequencies:
        entropy -= f * math.log(f, 2)
#     print(entropy)
    # entropy = label_num / total_counts #redo this!!!
#     print("the entropy is {0} with {1} edges, taking {2} bits".format(entropy, edges, bits_per_edge * edges * entropy))
    #this willl need to be changed in the future, assumes each label is 1 character long
#     print("there are {0} labels with less than {1} occurences, which use {2} bits".format(below_cutoff, frequency_cutoff, total_bits_below_cutoff))
    total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + total_bits_below_cutoff
    print("{0} total bits with a {1} frequency cutoff".format(int(total_bits), frequency_cutoff))

453 total bits with a 0 frequency cutoff
453 total bits with a 1 frequency cutoff
468 total bits with a 2 frequency cutoff
446 total bits with a 3 frequency cutoff
429 total bits with a 4 frequency cutoff
357 total bits with a 5 frequency cutoff


In [13]:
total_bits = edges * bits_per_edge + bits_per_edge * edges * entropy + 14 * below_cutoff
print(f"{total_bits} total bits from nodes+edges with a {frequency_cutoff} frequency cutoff")
#steve can you confirm this is correct?

260.0 total bits from nodes+edges with a 2 frequency cutoff


In [47]:
import math

#calculate codebook size
#use 2.6.2 and 2.6.3 in compact data structures book
#or just the canonical huffman wikipedia page is actually very helpful

#you can store a canonical huffman codebook as a list of symbols in order of mots frequent to least
#and then the number of symbols for each bit-length
#example: A = 11, B = 0, C = 101, D = 100
#becomes (1,1,2), ('B','A','C','D')
#so to calculate size we take the size of a list of ints (probably normal size maybe long? ask steve) of list size log base 2 of total symbols
#and add the length of a list of labels which is 7(n+1) for each label where n is label size
labels = hist.keys()
print("labels: ", list(labels))
label_bits = 0
for l in labels:
    label_bits += 7 * (len(l) + 1)
print("bits from labels: ", label_bits)
print("total labels: ", len(labels))
int_size = math.ceil(math.log(len(labels), 2)) #bit size of an unsigned int in c, goes up to 4,294,967,295, no idea if that is enough, ask steve
print(int_size)
int_size = math.ceil(math.log(int_size + 1, 2))
print(int_size)
bit_length_storage =  int_size * math.ceil(math.log(len(labels), 2))
print("codebook bit lengths storage size: ", bit_length_storage)
print("total codebook size: ", str(bit_length_storage + label_bits) + " bits")

labels:  ['k', 'i', 'h', 'w', 'u', 'e', 'a', 'c', 'g', '.', 'l', 'm', 'o']
bits from labels:  182
total labels:  13
4
3
codebook bit lengths storage size:  12
total codebook size:  194 bits


In [ ]:
# sweep with everything later